In [31]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pytorch imports

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

In [32]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, PIL
from glob import glob
import tensorflow as tf
from io import StringIO 
from PIL import Image
import pydot
import imageio as iio
import cv2

from __future__ import print_function
import pandas as pd
import shutil
import os
import sys

#import seaborn as sns
from sklearn import model_selection


import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import  img_to_array
from tensorflow.keras.preprocessing.image import array_to_img


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tqdm import tqdm

In [33]:
""" Sequential Model Architecture """
Sequential = tf.keras.models.Sequential

""" Data Preprocessing Functions """
Resizing = tf.keras.layers.experimental.preprocessing.Resizing
Rescaling = tf.keras.layers.experimental.preprocessing.Rescaling

""" Data Augmentation Functions """
RandomFlip = tf.keras.layers.experimental.preprocessing.RandomFlip
RandomRotation = tf.keras.layers.experimental.preprocessing.RandomRotation
RandomZoom = tf.keras.layers.experimental.preprocessing.RandomZoom

""" Artificial Neural Network Layer Inventory """
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout

""" Convolutional Neural Network Layer Inventory """
Conv2D = tf.keras.layers.Conv2D
MaxPool2D = tf.keras.layers.MaxPool2D
Flatten = tf.keras.layers.Flatten

""" Residual Network Layer Inventory """
ResNet50 = tf.keras.applications.resnet50.ResNet50

""" Function to Load Images from Target Folder """
get_image_data = tf.keras.utils.image_dataset_from_directory

In [34]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 100, 100

In [35]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

transformations = {
    'train': transforms.Compose([
        transforms.Resize((100,100)),
        transforms.CenterCrop((100,100)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'test': transforms.Compose([
        transforms.Resize((100,100)),
        transforms.CenterCrop((100,100)),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [36]:
learning_rate = 0.001
batch_size = 8
num_epochs = 5
num_classes = 2

# device
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(device)

cpu


In [37]:
DIR_PATH='C:\\Users\\SWAPNIL\\Desktop\\New folder\\'

In [38]:
#total_dataset = torchvision.datasets.ImageFolder(DIR_PATH,transform=transformations['train'])

#len(total_dataset),total_dataset[0][0].shape,total_dataset.class_to_idx

In [72]:
train ='C:\\Users\\SWAPNIL\\Desktop\\New folder\\Train1\\'
validation ='C:\\Users\\SWAPNIL\\Desktop\\New folder\\Dev1\\'
test ='C:\\Users\\SWAPNIL\\Desktop\\New folder\\Test1\\'
nb_train_samples =400
nb_validation_samples = 100
epochs = 4
batch_size = 20

In [40]:
train = torchvision.datasets.ImageFolder(train,transform=transformations['train'])

len(train),train[0][0].shape,train.class_to_idx

(20758, torch.Size([3, 100, 100]), {'fake': 0, 'real': 1})

In [41]:
val = torchvision.datasets.ImageFolder(validation,transform=transformations['train'])

len(val),val[0][0].shape,val.class_to_idx

(16987, torch.Size([3, 100, 100]), {'fake': 0, 'real': 1})

In [73]:
test = torchvision.datasets.ImageFolder(test,transform=transformations['test'])

len(test),test[0][0].shape,test.class_to_idx


(20843, torch.Size([3, 100, 100]), {'fake': 0, 'real': 1})

In [74]:
test_loader = DataLoader(dataset=test, batch_size=1, shuffle=True, num_workers=4)
print(len(test_loader))

20843


In [56]:
val_loader = DataLoader(dataset=val,
                       batch_size=1,
                       shuffle=True,
                       num_workers=4)
print(len(val_loader))

16987


In [43]:
# dataloaders
train_loader = DataLoader(dataset=train,
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=4)

val_loader = DataLoader(dataset=validation,
                       batch_size=1,
                       shuffle=True,
                       num_workers=4)
test_loader = DataLoader(dataset=test, batch_size=1, shuffle=True, num_workers=4)

In [44]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

1038
41
42


In [45]:
# testing dataloading 

examples = iter(train_loader)
samples,labels = examples.next()
print(samples.shape,labels.shape) # batch_size=8
len(train_loader)
len(val_loader)

torch.Size([20, 3, 100, 100]) torch.Size([20])


41

In [46]:
# custom CNN model class

class ConvNet(nn.Module):
    def __init__(self,model,num_classes):
        super(ConvNet,self).__init__()
        self.base_model = nn.Sequential(*list(model.children())[:-1]) # model excluding last FC layer
        self.linear1 = nn.Linear(in_features=2048,out_features=512)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=512,out_features=num_classes)
    
    def forward(self,x):
        x = self.base_model(x)
        x = torch.flatten(x,1)
        lin = self.linear1(x)
        x = self.relu(lin)
        out = self.linear2(x)
        return lin, out

In [47]:
model = torchvision.models.resnet50(pretrained=True) # base model

model = ConvNet(model,num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

In [48]:
print(model)

ConvNet(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(6

In [49]:
num_epochs=10

In [50]:
print(len(train_loader))

1038


In [51]:
# training loop

n_iters = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    for ii,(images,labels) in enumerate(train_loader):
        print(ii)
        images = images.to(device)
        labels = labels.to(device)
        
        _,outputs = model(images)
        loss = criterion(outputs,labels)
        
        # free_gpu_cache()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (ii+1)%108 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{ii+1}/{n_iters}], Loss = {loss.item():.6f}')
            
    print('----------------------------------------')
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
Epoch [1/10], Step [108/1038], Loss = 0.109810
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
Epoch [1/10], Step [216/1038], Loss = 0.095253
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253


In [52]:
# evaluating model and getting features of every image

def eval_model_extract_features(features,true_labels,model,dataloader,phase):

    with torch.no_grad():
        # for entire dataset
        n_correct = 0
        n_samples = 0

        model.eval()

        for images,labels in dataloader:

            images = images.to(device)
            labels = labels.to(device)

            true_labels.append(labels)
            
            ftrs,outputs = model(images)
            features.append(ftrs)

            _,preds = torch.max(outputs,1)
            n_samples += labels.size(0)
            n_correct += (preds == labels).sum().item()
                
        accuracy = n_correct/float(n_samples)

        print(f'Accuracy of model on {phase} set = {(100.0 * accuracy):.4f} %')

    return features,true_labels

In [53]:
features = []
true_labels = []

In [54]:
train_loader = DataLoader(dataset=train,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)

features,true_labels = eval_model_extract_features(features,true_labels,model,dataloader=train_loader,phase='training')

print(len(features),len(true_labels))

Accuracy of model on training set = 100.0000 %
20758 20758


In [57]:
features,true_labels = eval_model_extract_features(features,true_labels,model,dataloader=val_loader,phase='validation')

print(len(features),len(true_labels))

Accuracy of model on validation set = 84.7766 %
37745 37745


In [58]:
ftrs = features.copy() 
lbls = true_labels.copy()

In [59]:
for i in range(len(ftrs)):
    ftrs[i]=ftrs[i].cpu().numpy()

ftrs[0].shape

(1, 512)

In [60]:
for i in range(len(lbls)):
    lbls[i]=lbls[i].cpu().numpy()

lbls[0].shape

(1,)

In [61]:
type(ftrs),type(lbls)

(list, list)

In [62]:
ftrs = np.array(ftrs)
lbls = np.array(lbls)

ftrs.shape,lbls.shape

((37745, 1, 512), (37745, 1))

In [63]:
n_samples = ftrs.shape[0]*ftrs.shape[1]
n_features = ftrs.shape[2]
ftrs = ftrs.reshape(n_samples,n_features)

print(ftrs.shape)

(37745, 512)


In [64]:
n_lbls = lbls.shape[0]
lbls = lbls.reshape(n_lbls)

print(lbls.shape)

(37745,)


In [65]:
# save to csv
ftrs_df = pd.DataFrame(ftrs)
ftrs_df.to_csv('Protocol1_features.csv',index=False)

# reloading the saved csv into a df

ftrs_df = pd.read_csv('Protocol1_features.csv')
ftrs_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.196656,0.131031,0.085500,0.444914,1.501654,-0.431384,-0.075061,-0.413970,-0.332213,-0.671525,...,0.380323,-0.173760,-0.034144,0.296806,-0.337709,0.532591,-0.160591,1.799268,0.383572,-0.610647
1,-0.186134,-0.420871,0.057183,0.205021,2.235867,-0.751498,-0.284065,-0.255555,-0.438462,-0.472919,...,0.121393,-0.174453,-0.472127,0.403736,-0.314814,0.159944,-0.120569,2.412055,0.041659,-0.548541
2,0.114465,0.056228,-0.092826,0.494534,1.481885,-0.417989,-0.036052,-0.326369,-0.296863,-0.595545,...,0.300450,-0.142185,0.013301,0.409179,-0.313531,0.514522,-0.064553,1.652749,0.461697,-0.545460
3,0.227946,-0.150725,0.183412,0.326855,1.342035,-0.399909,-0.253330,-0.082883,-0.136401,-0.670254,...,0.282830,0.018894,0.012146,0.296690,-0.340916,0.177460,-0.175406,1.567440,0.253403,-0.352246
4,-0.153753,-0.554442,0.075598,0.262754,2.219750,-0.874376,-0.287445,-0.261032,-0.194963,-0.591691,...,0.155271,-0.117164,-0.331512,0.360172,-0.401951,0.157942,-0.039321,2.489741,-0.024143,-0.455594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37740,-0.042379,-0.252940,0.317056,0.100085,1.563200,-0.142150,0.019624,0.083130,-0.648777,-0.642947,...,-0.022598,0.333571,-0.311443,-0.068994,-0.298075,0.135606,-0.385846,1.622618,-0.242379,-0.178022
37741,0.085589,0.706391,0.519276,0.057886,1.689012,0.025365,0.108554,0.285164,-0.224530,0.251015,...,-0.016441,0.479818,0.563576,-0.036037,-0.241451,0.497109,-0.262044,1.098564,0.282257,-0.294577
37742,0.230872,-0.055465,0.747793,0.105292,2.044259,-0.407330,-0.452705,-0.154725,-1.054052,0.818073,...,0.413436,0.554315,-0.621148,-0.019415,-0.770542,-0.018916,-0.913274,1.900150,0.167431,-0.519528
37743,0.363489,-0.296895,0.090230,0.356263,2.173729,-0.436008,-0.113486,0.010616,-0.266769,0.017123,...,-0.203507,0.510521,-0.374180,-0.100003,-0.427096,0.386292,-0.375458,1.856625,-0.388228,-0.404056


In [66]:
# appending labels to the feature set
ftrs_df['label'] = lbls

ftrs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
0,0.196656,0.131031,0.085500,0.444914,1.501654,-0.431384,-0.075061,-0.413970,-0.332213,-0.671525,...,-0.173760,-0.034144,0.296806,-0.337709,0.532591,-0.160591,1.799268,0.383572,-0.610647,0
1,-0.186134,-0.420871,0.057183,0.205021,2.235867,-0.751498,-0.284065,-0.255555,-0.438462,-0.472919,...,-0.174453,-0.472127,0.403736,-0.314814,0.159944,-0.120569,2.412055,0.041659,-0.548541,0
2,0.114465,0.056228,-0.092826,0.494534,1.481885,-0.417989,-0.036052,-0.326369,-0.296863,-0.595545,...,-0.142185,0.013301,0.409179,-0.313531,0.514522,-0.064553,1.652749,0.461697,-0.545460,0
3,0.227946,-0.150725,0.183412,0.326855,1.342035,-0.399909,-0.253330,-0.082883,-0.136401,-0.670254,...,0.018894,0.012146,0.296690,-0.340916,0.177460,-0.175406,1.567440,0.253403,-0.352246,0
4,-0.153753,-0.554442,0.075598,0.262754,2.219750,-0.874376,-0.287445,-0.261032,-0.194963,-0.591691,...,-0.117164,-0.331512,0.360172,-0.401951,0.157942,-0.039321,2.489741,-0.024143,-0.455594,0


In [67]:
ftrs_df.to_csv('olunpu_protocol1.csv',index=False)

print('feature set saved successfully !')

feature set saved successfully !


In [68]:
# save model
MODEL_PATH = 'oulunpu_protocol1.pth'
torch.save(model.state_dict(),MODEL_PATH)

In [75]:
features,true_labels = eval_model_extract_features(features,true_labels,model,dataloader=test_loader,phase='testing')

print(len(features),len(true_labels))

Accuracy of model on testing set = 85.3188 %
58588 58588
